In [ ]:
%pip install astropy

In [31]:
# 
#
#
import copy
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import chirp, find_peaks, peak_widths, peak_prominences
from scipy import interpolate

In [3]:
#
# https://github.com/ZwickyTransientFacility/ztf_information/blob/master/field_grid/ZTF_Fields.txt
#
filename = '/media3/majumder/zwaad/zwad/slsn_data/ZTF_Fields.txt'
#
infile = open(filename, 'r')
lines = infile.readlines()
fields = np.ones((1778, 9))
ind=0
#
#
#
for l in range(1,len(lines)):
    
    line = lines[l]
    sline = line.split()
    fields[ind,:] = sline
    ind+=1
infile.close()  
#
#
#
df = pd.DataFrame(fields, columns = ['ID', 'RA', 'DEC', 'Ebv', 'Gal_long', 'Gal_lat', 'Ecl_Long', 'Ecl_Lat', 'Entry'])
#
#
#
df_cut = df.loc[(df['Gal_lat']>= 30.00)]

In [ ]:
#
#
#
oid = np.memmap('/media3/majumder/zwaad/zwad/slsn_data/oid_short_r_300.dat', mode='r+', dtype=np.uint64)
oid_sort = oid.astype(int)
#
#
#
with open('/media3/majumder/zwaad/zwad/slsn_data/feature_short_r_300.name') as f:
    names = f.read().split()
#
#
#
dtype = [(name, np.float32) for name in names]
#
#
#
feature = np.memmap('/media3/majumder/zwaad/zwad/slsn_data/feature_short_r_300.dat', mode='r+', dtype=dtype, shape=oid.shape)
# 
#
#
np.savetxt('/media3/majumder/zwaad/zwad/slsn_data/oid_my.txt', oid_sort)

In [ ]:
#
#
#
a = np.array(np.loadtxt('/media3/majumder/zwaad/zwad/slsn_data/oid_my.txt'))
#
#
#
a_first = a//10**(12)
uniq = np.unique(a_first)
common = np.intersect1d(uniq, df_cut['ID'])
feature_fl = np.array(feature)
#
#
#
mask = np.isin(a_first, common)
oid_new = oid*mask
oid_n = oid_new[oid_new != 0]
#
#
#
n = 0
feature_n = np.array(np.ones((len(oid_n))),  dtype = object)
#
for i in range(len(oid_new)):
    if mask[i]:
        feature_n[n] = feature_fl[i]
        n+=1
    else:
        continue



In [6]:
print(len(oid_n))
print(len(feature_n))

13761212
13761212


In [10]:
#
#
#
oid_save = np.memmap('/media3/majumder/zwaad/zwad/slsn_data/oid_cut30_short_r_300.dat', mode='w+', dtype=np.uint64, shape=oid_n.shape)
#
#
#
with open('/media3/majumder/zwaad/zwad/slsn_data/feature_short_r_300.name') as f:
    names = f.read().split()
#
#
#
dtype = [(name, np.float32) for name in names]
feature_save = np.memmap('/media3/majumder/zwaad/zwad/slsn_data/feature_cut30_short_r_300.dat', mode='w+', dtype=dtype, shape=oid_n.shape)

In [11]:
oid_save[:] = oid_n[:]

In [ ]:
oid_open = np.memmap('/media3/majumder/zwaad/zwad/slsn_data/oid_cut30_short_r_300.dat', mode='r', dtype=np.uint64)
print(oid_open.shape, oid_open[0:10])

In [16]:
feature_save[:] = feature_n[:]

In [ ]:
feature_open = np.memmap('/media3/majumder/zwaad/zwad/slsn_data/feature_cut30_short_r_300.dat', mode='r', dtype=dtype, shape=oid_n.shape)
print(len(feature_open))

13761212
